In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

In [2]:
dfh18 = pd.read_csv("500_Cities__Local_Data_for_Better_Health__2018_release.csv")

In [4]:
#Display dataframe
display(dfh18.head())
display(dfh18.info())

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
0,2016,US,United States,NaN,US,BRFSS,Prevention,59,Current lack of health insurance among adults ...,%,...,14.9,NaN,NaN,308745538,NaN,PREVENT,ACCESS2,NaN,NaN,Health Insurance
1,2016,US,United States,NaN,US,BRFSS,Prevention,59,Current lack of health insurance among adults ...,%,...,11.8,NaN,NaN,308745538,NaN,PREVENT,ACCESS2,NaN,NaN,Health Insurance
2,2016,US,United States,NaN,US,BRFSS,Health Outcomes,59,Arthritis among adults aged >=18 Years,%,...,23.2,NaN,NaN,308745538,NaN,HLTHOUT,ARTHRITIS,NaN,NaN,Arthritis
3,2016,US,United States,NaN,US,BRFSS,Health Outcomes,59,Arthritis among adults aged >=18 Years,%,...,25.6,NaN,NaN,308745538,NaN,HLTHOUT,ARTHRITIS,NaN,NaN,Arthritis
4,2016,US,United States,NaN,US,BRFSS,Unhealthy Behaviors,59,Binge drinking among adults aged >=18 Years,%,...,18.2,NaN,NaN,308745538,NaN,UNHBEH,BINGE,NaN,NaN,Binge Drinking


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 810103 entries, 0 to 810102
Data columns (total 24 columns):
Year                          810103 non-null int64
StateAbbr                     810103 non-null object
StateDesc                     810103 non-null object
CityName                      810047 non-null object
GeographicLevel               810103 non-null object
DataSource                    810103 non-null object
Category                      810103 non-null object
UniqueID                      810103 non-null object
Measure                       810103 non-null object
Data_Value_Unit               810103 non-null object
DataValueTypeID               810103 non-null object
Data_Value_Type               810103 non-null object
Data_Value                    787311 non-null float64
Low_Confidence_Limit          787311 non-null float64
High_Confidence_Limit         787311 non-null float64
Data_Value_Footnote_Symbol    22794 non-null object
Data_Value_Footnote           22794 non-

None

In [4]:
#Filter for desired columns, remove redundant columns and rename columns
dfh18 = dfh18[['Year', 'GeographicLevel', 'StateDesc', 'CityName','Short_Question_Text','Measure','Data_Value_Type', 
               'Data_Value','Low_Confidence_Limit', 'High_Confidence_Limit','PopulationCount','GeoLocation',
               'CityFIPS','TractFIPS']]
dfh18.columns = ["Year","GeographicLevel","State","City","Category","Measure","Data_Type",
                  "Data_Value","Low_Confidence_Limit","High_Confidence_Limit","Population","Tract GeoLocation",
                  "CityFIPS","TractFIPS"]

In [5]:
# Filter out rows at the US or City level
dfh18 = dfh18.loc[dfh18["GeographicLevel"] == "Census Tract"]

In [6]:
#Filter values related to heart disease
dfh18 = dfh18.loc[dfh18["Measure"].isin(['High blood pressure among adults aged >=18 Years',
                                         'Coronary heart disease among adults aged >=18 Years',
                                         'High cholesterol among adults aged >=18 Years who have been screened in the past 5 Years',
                                         'Current smoking among adults aged >=18 Years',
                                         'Diagnosed diabetes among adults aged >=18 Years',
                                         'No leisure-time physical activity among adults aged >=18 Years',
                                         'Obesity among adults aged >=18 Years'])]

In [7]:
# Remove rows with no data value
dfh18 = dfh18.loc[dfh18.Data_Value.notnull()]

In [8]:
# Store values in separate variables
smoking = dfh18.loc[dfh18["Category"] == "Current Smoking"]
hypertension = dfh18.loc[dfh18["Category"] == "High Blood Pressure"]
obesity = dfh18.loc[dfh18["Category"] == "Obesity"]
sedentery = dfh18.loc[dfh18["Category"] == "Physical Inactivity"]
cholesterol = dfh18.loc[dfh18["Category"] == "High Cholesterol"]
diabetes = dfh18.loc[dfh18["Category"] == "Diabetes"]
heartd = dfh18.loc[dfh18["Category"] == "Coronary Heart Disease"]

In [9]:
# Read zip-to-tract conversion file
tractzip = pd.read_excel("Tract-zip.xlsx")

# Extract desired columns
tractzip = tractzip[["zip","tract","res_ratio"]]
display(tractzip.head())

,zip,tract,res_ratio
0,36067,1001020100,1.000000
1,36067,1001020200,1.000000
2,36067,1001020300,1.000000
3,36067,1001020400,0.025014
4,36066,1001020400,0.974986


In [10]:
# Group by tracts and select rows with the greatest resident percentage
tractgroups = tractzip.groupby("tract")["res_ratio"].max().reset_index()

# Merge tract groups with the original df containing zip codes
# resulting in one zip code per tract
tractzipm = pd.merge(tractzip,tractgroups, how="inner")
tractzipm.columns = ["Zip Code","tract","res_ratio"]
tractzipm = tractzipm[["Zip Code","tract"]]

display(tractzipm.head())
display(tractzipm.info())

,Zip Code,tract
0,36067,1001020100
1,36067,1001020200
2,36067,1001020300
3,36066,1001020400
4,36066,1001020500


<class 'pandas.core.frame.DataFrame'>
Int64Index: 73659 entries, 0 to 73658
Data columns (total 2 columns):
Zip Code    73659 non-null int64
tract       73659 non-null int64
dtypes: int64(2)
memory usage: 1.7 MB


None

In [11]:
# Merge health values with conversion dataframe
smokingm = pd.merge(smoking,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
hypertensionm = pd.merge(hypertension,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
obesitym = pd.merge(obesity,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
sedenterym = pd.merge(sedentery,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
cholesterolm = pd.merge(cholesterol,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
diabetesm = pd.merge(diabetes,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
heartdm = pd.merge(heartd,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")

In [12]:
# Group by zip code for each value and calculate weighted mean prevalence
smokingmp = smokingm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
smokingmp.columns = ["Zip Code","Smoking Prevalence"]

hypertensionmp = hypertensionm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
hypertensionmp.columns = ["Zip Code","Hypertension Prevalence"]

obesitymp = obesitym.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
obesitymp.columns = ["Zip Code","Obesity Prevalence"]

sedenterymp = sedenterym.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
sedenterymp.columns = ["Zip Code","Sedentarism Prevalence"]

cholesterolmp = cholesterolm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
cholesterolmp.columns = ["Zip Code","Cholesterol Prevalence"]

diabetesmp = diabetesm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
diabetesmp.columns = ["Zip Code","Diabetes Prevalence"]

heartdmp = heartdm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
heartdmp.columns = ["Zip Code","Heart Disease Prevalence"]

#Merge the values
_ = pd.merge(smokingmp,hypertensionmp, on="Zip Code")
_ = pd.merge(_,obesitymp, on="Zip Code")
_ = pd.merge(_,sedenterymp, on="Zip Code")
_ = pd.merge(_,cholesterolmp, on="Zip Code")
_ = pd.merge(_,diabetesmp, on="Zip Code")
datameans = pd.merge(_,heartdmp, on="Zip Code")

display(datameans.head())
display(datameans.info())

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence
0,1101,28.300000,36.400000,44.000000,46.600000,41.300000,19.300000,9.300000
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889
4,1108,23.844497,30.236068,35.381122,34.251449,35.500119,12.581968,6.943035


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4793 entries, 0 to 4792
Data columns (total 8 columns):
Zip Code                    4793 non-null int64
Smoking Prevalence          4793 non-null float64
Hypertension Prevalence     4793 non-null float64
Obesity Prevalence          4793 non-null float64
Sedentarism Prevalence      4793 non-null float64
Cholesterol Prevalence      4793 non-null float64
Diabetes Prevalence         4793 non-null float64
Heart Disease Prevalence    4793 non-null float64
dtypes: float64(7), int64(1)
memory usage: 337.0 KB


None

In [13]:
#Open and import the fast food csv into a dataframe
dfff = pd.read_csv("fastfoodmaps_locations_2007.csv", header=None)
dfff.head()

,0,1,2,3,4,5,6,7,8,9
0,1,b,3601 N.W. 27th Avenue,Miami,FL,33142,(305) 638-3838,25.8092,-80.2400,0
1,2,b,8995 N. W. 7th Avenue,Miami,FL,33150,(305) 754-8453,25.8587,-80.2094,0
2,3,b,30390 South Dixie Highway,Homestead,FL,33030,(305) 247-7181,25.4849,-80.4610,0
3,4,b,7975 N. W. 27th Avenue,Miami,FL,33147,(305) 836-8152,25.8471,-80.2415,0
4,5,b,9201 South Dixie Highway,Miami,FL,33156,(305) 666-1130,25.6849,-80.3125,0


In [14]:
#Select and rename useful columns
dfff = dfff[[1,3,4,5,7,8]]
dfff.columns = ["Restaurant Count","City","State","Zip Code","Latitude","Longitude"]

# Map restaurant names to the Name column
dfff["Restaurant Count"] = 1
display(dfff.head())

display(dfff.info())

,Restaurant Count,City,State,Zip Code,Latitude,Longitude
0,1,Miami,FL,33142,25.8092,-80.2400
1,1,Miami,FL,33150,25.8587,-80.2094
2,1,Homestead,FL,33030,25.4849,-80.4610
3,1,Miami,FL,33147,25.8471,-80.2415
4,1,Miami,FL,33156,25.6849,-80.3125


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50002 entries, 0 to 50001
Data columns (total 6 columns):
Restaurant Count    50002 non-null int64
City                50002 non-null object
State               50002 non-null object
Zip Code            50002 non-null object
Latitude            50002 non-null float64
Longitude           50002 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 2.3+ MB


None

In [15]:
#Display zip code values
dfff["Zip Code"].sort_values()

36256         01001
26731         01007
37335         01008
37016         01008
37332         01008
37015         01008
37017         01011
35629         01013
19504         01020
31419         01020
49204         01020
39085         01020
26697    01020-3964
1987          01027
25669         01027
20898         01028
31740         01035
442           01035
24022         01035
47428         01035
14134         01035
28927         01040
20802         01040
49873         01040
30265         01040
33139         01040
1912          01040
37328         01056
2454          01056
32356         01056
            ...    
39760         99654
3023          99654
7439          99654
20167         99654
26279         99654
15848         99654
35796    99654-8105
32669         99669
41956         99669
48916         99687
15917         99701
43272         99701
12207         99701
23010         99701
31759    99701-1706
28184    99701-4092
3692          99702
3939          99703
40387         99705


In [16]:
# Remove trailing digits for zip codes
nohyphens = []
for x in dfff["Zip Code"]:
    nohyphens.append(x.replace('-',''))
dfff["Zip Code"] = nohyphens

zclist = []
for zc in dfff["Zip Code"]:
    if len(zc)>5:
        zclist.append(zc[:6])
    else:
        zclist.append(zc)
dfff["Zip Code"] = zclist
dfff["Zip Code"] = dfff["Zip Code"].astype('int')

In [17]:
# Group by zip code and get a count of restaurants
resgroups = dfff.groupby("Zip Code")["Restaurant Count"].sum().reset_index()
resgroups.head()

,Zip Code,Restaurant Count
0,1001,1
1,1007,1
2,1008,4
3,1011,1
4,1013,1


In [18]:
# Merge health dataframe with fast food location dataframe
datameansm = pd.merge(datameans,resgroups,how="left",on="Zip Code")

# Fill NaN values with 0
datameansm = datameansm.fillna(0)

display(datameansm.head())
display(datameansm.info())
display(datameansm.describe())

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count
0,1101,28.300000,36.400000,44.000000,46.600000,41.300000,19.300000,9.300000,2.0
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,0.0
4,1108,23.844497,30.236068,35.381122,34.251449,35.500119,12.581968,6.943035,2.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4793 entries, 0 to 4792
Data columns (total 9 columns):
Zip Code                    4793 non-null int64
Smoking Prevalence          4793 non-null float64
Hypertension Prevalence     4793 non-null float64
Obesity Prevalence          4793 non-null float64
Sedentarism Prevalence      4793 non-null float64
Cholesterol Prevalence      4793 non-null float64
Diabetes Prevalence         4793 non-null float64
Heart Disease Prevalence    4793 non-null float64
Restaurant Count            4793 non-null float64
dtypes: float64(8), int64(1)
memory usage: 374.5 KB


None

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count
count,4793.000000,4793.000000,4793.000000,4793.000000,4793.000000,4793.000000,4793.000000,4793.000000,4793.000000
mean,59454.246192,18.070710,30.123836,30.065707,24.866540,34.337529,10.456542,5.748557,3.782808
std,29133.980164,5.564226,7.727673,7.646405,8.595184,5.086517,3.930864,1.926900,3.715321
min,1101.000000,2.470610,5.700000,10.519778,8.300000,8.500000,0.800000,0.500000,0.000000
25%,33417.000000,14.007323,25.325989,24.532893,18.115248,31.836474,7.763077,4.515044,1.000000
50%,66103.000000,17.443807,29.165720,29.300000,23.558648,34.772317,9.771208,5.628687,3.000000
75%,85301.000000,21.600000,34.028414,34.822722,30.632701,37.499166,12.500000,6.886213,6.000000
max,99577.000000,40.900000,69.300000,56.000000,52.308167,52.100000,28.158736,21.000000,21.000000


In [5]:
# Import uszipcode to retrieve income for zip codes
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
zipstats = search.by_median_household_income(lower=-1, upper=2147483648, zipcode_type='Standard', 
                                            sort_by='median_household_income', ascending=False, returns=999999)

In [20]:
# Transform list of dictionaries to dataframe
list = []
for entry in zipstats:
    list.append(entry.to_dict())

zipstats_df = pd.DataFrame(list)
display(zipstats_df.head())

,area_code_list,bounds_east,bounds_north,bounds_south,bounds_west,common_city_list,county,housing_units,land_area_in_sqmi,lat,...,occupied_housing_units,population,population_density,post_office_city,radius_in_miles,state,timezone,water_area_in_sqmi,zipcode,zipcode_type
0,[970],-108.683032,37.573355,37.435026,-108.849361,[Yellow Jacket],Montezuma County,64,27.12,37.540,...,54,131,5.0,"Yellow Jacket, CO",2.000000,CO,Mountain,0.00,81335,Standard
1,"[301, 410, 443]",-76.529878,39.040506,39.017669,-76.570564,"[Annapolis, Sherwood Forest, Sherwood Frst]",Anne Arundel County,340,0.99,39.030,...,203,544,549.0,"Annapolis, MD",0.965909,MD,Eastern,0.75,21405,Standard
2,[503],-121.679473,45.366297,45.215218,-121.872530,"[Government Camp, Government Cp, Timberline Lo...",Clackamas County,713,38.01,45.300,...,114,217,6.0,"Government Camp, OR",2.000000,OR,Pacific,0.10,97028,Standard
3,"[281, 346, 713, 832]",-95.355302,29.757155,29.749765,-95.364051,[Houston],Harris County,471,0.12,29.754,...,249,366,2965.0,"Houston, TX",0.284091,TX,Central,0.00,77010,Standard
4,[307],-105.793154,41.128117,40.514119,-106.195438,"[Jelm, Laramie]",Albany County,170,384.84,41.060,...,54,100,0.0,"Jelm, WY",6.000000,WY,Mountain,0.85,82063,Standard


In [21]:
# Select zipcode and income columns and rename columns
zipincome = zipstats_df[["zipcode","median_household_income","population"]]
zipincome.columns = ["Zip Code","median_household_income","Population"]

# Convert zip codes to integers
zipincome["Zip Code"] = zipincome["Zip Code"].astype("int")

display(zipincome.head())
display(zipincome.info())

,Zip Code,median_household_income,Population
0,81335,250001,131
1,21405,250001,544
2,97028,250001,217
3,77010,250001,366
4,82063,250001,100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28847 entries, 0 to 28846
Data columns (total 3 columns):
Zip Code                   28847 non-null int32
median_household_income    28847 non-null int64
Population                 28847 non-null int64
dtypes: int32(1), int64(2)
memory usage: 563.5 KB


None

In [22]:
# Merge income data with dataframe
_ = pd.merge(datameansm,zipincome,how="left",on="Zip Code")

# Drop null income and population values
_ = _.dropna()

display(_.head())
display(_.info())
display(_.describe())
data = _.copy()

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,median_household_income,Population
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0,32273.0,22865.0
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0,18402.0,12350.0
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,0.0,21737.0,11611.0
4,1108,23.844497,30.236068,35.381122,34.251449,35.500119,12.581968,6.943035,2.0,34064.0,26688.0
5,1109,23.329629,32.285360,37.212078,33.779183,34.154532,13.474276,6.706302,2.0,33376.0,30250.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4396 entries, 1 to 4792
Data columns (total 11 columns):
Zip Code                    4396 non-null int64
Smoking Prevalence          4396 non-null float64
Hypertension Prevalence     4396 non-null float64
Obesity Prevalence          4396 non-null float64
Sedentarism Prevalence      4396 non-null float64
Cholesterol Prevalence      4396 non-null float64
Diabetes Prevalence         4396 non-null float64
Heart Disease Prevalence    4396 non-null float64
Restaurant Count            4396 non-null float64
median_household_income     4396 non-null float64
Population                  4396 non-null float64
dtypes: float64(10), int64(1)
memory usage: 412.1 KB


None

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,median_household_income,Population
count,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000
mean,58938.053230,17.933767,30.093603,29.941820,24.607667,34.361576,10.378505,5.713612,4.113512,55588.992266,29072.211328
std,29075.072266,5.464079,7.466103,7.514914,8.406547,4.747407,3.799013,1.835993,3.702730,24451.268427,17422.209449
min,1104.000000,2.470610,7.484369,10.519778,8.300000,9.500000,1.200000,0.700000,0.000000,6067.000000,219.000000
25%,33408.500000,13.930448,25.375412,24.515797,18.093263,31.886738,7.753140,4.517160,1.000000,38269.500000,16280.750000
50%,64127.500000,17.352454,29.100000,29.196297,23.345756,34.706569,9.698135,5.605186,3.000000,51148.000000,26748.000000
75%,85140.250000,21.358638,33.869172,34.564970,30.246419,37.384407,12.308810,6.800000,6.000000,68416.500000,38721.750000
max,99577.000000,36.400000,69.300000,53.355085,52.308167,52.100000,28.158736,21.000000,21.000000,230952.000000,113916.000000


In [24]:
# Create a new column specifying population group by size
popgroups = []
for pop in data["Population"]:
    if pop >80000:
        popgroups.append(">80000")
    if pop <20000:
        popgroups.append("<20000")
    if pop in range(20000,40000):
        popgroups.append("20000-40000")
    if pop in range(40000,60000):
        popgroups.append("40000-60000")
    if pop in range(60000,80000):
        popgroups.append("60000-80000")
        
data["Population Group"]= popgroups
data.head()

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,median_household_income,Population,Population Group
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0,32273.0,22865.0,20000-40000
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0,18402.0,12350.0,<20000
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,0.0,21737.0,11611.0,<20000
4,1108,23.844497,30.236068,35.381122,34.251449,35.500119,12.581968,6.943035,2.0,34064.0,26688.0,20000-40000
5,1109,23.329629,32.285360,37.212078,33.779183,34.154532,13.474276,6.706302,2.0,33376.0,30250.0,20000-40000


In [25]:
# Create a new column specifying restaurant group by sum
restgroups = []
for count in data["Restaurant Count"]:
    if count in range(0,8):
        restgroups.append("0-7")
    if count in range(8, 22):
        restgroups.append("8-21")
        
data["Restaurant Group"]= restgroups
data.head()

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,median_household_income,Population,Population Group,Restaurant Group
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0,32273.0,22865.0,20000-40000,0-7
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0,18402.0,12350.0,<20000,0-7
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,0.0,21737.0,11611.0,<20000,0-7
4,1108,23.844497,30.236068,35.381122,34.251449,35.500119,12.581968,6.943035,2.0,34064.0,26688.0,20000-40000,0-7
5,1109,23.329629,32.285360,37.212078,33.779183,34.154532,13.474276,6.706302,2.0,33376.0,30250.0,20000-40000,0-7


In [26]:
# Export to file
export_csv = data.to_csv('heart_disease_restaurants.csv', header=True)